In [1]:
%%capture
!pip install sentence-transformers

In [2]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import wandb
wandb.login()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sbastola/muril-base-cased-sentence-transformer-snli-nepali-2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/165 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.15k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [5]:
import csv

def read_csv(filename):
    data = []
    with open(filename, 'r', newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip header row if it exists
        for row in reader:
            data.append([row[1], row[2]])
    return data

filename = '/kaggle/input/sentence-compression-nepali/train.csv'  # Replace 'your_dataset.csv' with the path to your CSV file
dataset = read_csv(filename)

len(dataset)

180000

In [6]:
print(f"Example data: {dataset[8]}")

Example data: [' सेन बाराक ओबामा र सेन हिलारी क्लिन्टनका पूर्व राष्ट्रपतिका प्रतिद्वन्द्वी गभर्नर बिल रिचर्डसनले आज ओबामालाई समर्थन गरेका छन्।', ' बिल रिचर्डसनले ओबामालाई समर्थन गरे']


In [7]:
train_dataset = []

for i in range(int(len(dataset) * 0.3)):
    train_dataset.append(dataset[i])
    
len(train_dataset)

54000

In [8]:
for i in range(5):
    print(train_dataset[i])

[' भुवनेश्वर, फेब्रुअरी 15 विश्व हिन्दू परिषदका नेता अशोक सिंघलले सोमबार भने कि 2008 को कंधमाल दंगाको कारण स्वामी लक्ष्मणानन्दको हत्या मिडियामा प्रक्षेपण गरे अनुसार वीएचपी नेता थिएनन्।', ' ‘लक्ष्मनानन्द सञ्चारमाध्यममा प्रक्षेपण गरेजस्तो विहिप नेता होइनन्’']
[" 'पर्पल रेन' हिटमेकर प्रिन्सलाई गत वर्ष प्रदान गरिएको भुक्तान नगरिएको सेवाहरूको लागि भिडियो सम्पादकलाई $ 57,600 तिर्न आदेश दिइएको छ।", ' प्रिन्सले भिडियो सम्पादकलाई प्रदान गरिएका सेवाहरूको लागि $ 57,600 तिर्नेछ']
[' जनवरी सम्म, एयर क्यानाडाले आफ्नो क्यानेडियन, क्यारिबियन र मेक्सिकन उडानहरूमा दोस्रो चेक गरिएको झोलाको लागि यात्रुहरूलाई $ 20 चार्ज गर्नेछ।', ' एयर क्यानाडाले दोस्रो चेक गरिएको झोलाको लागि $ 20 चार्ज गर्नेछ']
[' यो समय हो कि कसैले मोटर वाहन ट्राफिकको नियमनको जिम्मा लिने व्यक्तिहरूको ध्यानमा ल्याउँदछ कि फिट्जगेराल्ड स्ट्रीट, उत्तर पर्थमा स्पीड कुशनहरू निराशाजनक विफलता हुन्।', ' गति कुशन एक विफलता हो']
['रेड सोक्स पिचर टिम वेकफिल्डले फोर्ट मायर्स, फ्लामा रहेको टोलीको नयाँ वसन्त प्रशिक्षण परिसरमा पत्रकार सम्मेलनको क्रममा आ

In [9]:
from sentence_transformers import InputExample

train_examples = []

for i in range(len(train_dataset)):
  example = train_dataset[i]
  train_examples.append(InputExample(texts=[example[0], example[1]]))

len(train_examples)

54000

In [10]:
from torch.utils.data import DataLoader
from sentence_transformers import losses

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss = losses.MultipleNegativesRankingLoss(model=model)
num_epochs = 5
warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data

warmup_steps

1687

In [11]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=num_epochs,
          warmup_steps=warmup_steps) 

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3375 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3375 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3375 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3375 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3375 [00:00<?, ?it/s]

In [12]:
model.save_to_hub(
    "sentence-transformer-nepali", 
    organization="Yunika",
    exist_ok=True, 
    )

model.safetensors:   0%|          | 0.00/950M [00:00<?, ?B/s]

'https://huggingface.co/Yunika/sentence-transformer-nepali/commit/486ba43d418c4034d98588abe47dcf6f6f36d06e'